# Covid-19

In [ ]:
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import optparse
import subprocess
import random

import numpy as np

import sqlite3

import csv

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn import decomposition

import pygraphviz

from networkx.algorithms import community

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
conn = sqlite3.connect('database.db')
conn.text_factory = str
c = conn.cursor()

In [ ]:
c.execute("DROP TABLE IF EXISTS basic_information")
conn.commit()
c.execute('''CREATE TABLE basic_information(
 PID int,
 AGE integer
)''')

c.execute("CREATE INDEX basicinfo_pid on basic_information (PID)")

In [ ]:
files = ["data/dataset.csv"]
for filename in files:
   with open(filename, mode='r') as csv_file:
         csv_reader = csv.DictReader(csv_file)
         for row in csv_reader:
            c.execute("SELECT PID FROM basic_information WHERE PID = '" +row["Patient ID"] + "'")
            if c.fetchone() == None:
               c.execute("INSERT INTO basic_information(PID, AGE) VALUES(?, ?)", (row["Patient ID"], row["Patient age quantile"]))
conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM basic_information", conn)

In [ ]:
c.execute("DROP TABLE IF EXISTS exams")
conn.commit()
c.execute('''CREATE TABLE exams(
    PID int, 
    SARSCov2 text,
    Hematocrit text,
    Hemoglobin text,
    Platelets text,
    mean_platelet_volume text, 
    red_blood_cells text,
    lymphocytes text,
    mean_corpuscular_hemoglobin_concentration text,
    leukocytes text,
    basophils text,
    mean_corpuscular_hemoglobi text,
    eosinophils text,
    mean_corpuscular_volume text,
    monocytes text,
    red_blood_cell_distribution text,
    serum_glucose text,
    respiratory_syncytial_virus text,
    influenza_aa text,
    influenza_bb text,
    parainfluenza_1 text,
    coronavirusNL63 text,
    rhinovirus text,
    mycoplasma_pneumoniae text,
    coronavirus_HKU1 text,
    parainfluenza_3 text,
    chlamydophila_pneumoniae text,
    adenovirus text,
    parainfluenza_4 text,
    coronavirus229E text,
    coronavirusOC43 text,
    inf_A_H1N1_2009 text,
    bordetella_pertussis text,
    metapneumovirus text,
    parainfluenza_2 text,
    neutrophils text,
    urea text,
    proteina_C_reativa text,
    creatinine text,
    potassium text,
    sodium text,
    influenza_B text,
    influenza_A text,
    alanine_transaminase text,
    aspartate_transaminase text,
    gamma_glutamyltransferase text,
    total_bilirubin text,
    direct_bilirubin text,
    indirect_bilirubin text,
    alkaline_phosphatase text,
    ionized_calcium text,
    strepto_A text,
    magnesium text,
    pCO2_v text,
    Hb_saturation_v text,
    base_excess_v text,
    pO2_v text,
    Fio2 text,
    total CO2 text,
    pH text,
    HCO3_v text,
    Rods text,
    Segmented text,
    Promyelocytes text,
    Metamyelocytes text,
    Myelocytes text,
    Myeloblasts text,
    Urine_Esterase text,
    Urine_Aspect text,
    Urine_pH text,
    Urine_Hemoglobin text,
    Urine_Bile_pigments text,
    Urine_Ketone_Bodies text,
    Urine_Nitrite text,
    Urine_Density text,
    Urine_Urobilinogen text,
    Urine_Protein text,
    Urine_Sugar text,
    Urine_Leukocytes text,
    Urine_Crystals text,
    Urine_Red_blood_cells text,
    Urine_Hyaline_cylinders text,
    Urine_Granular_cylinders text,
    Urine_Yeasts text,
    Urine_Color text,
    Partial_thromboplastin text,
    Relationship text,
    International text,
    Lactic_Dehydrogenase text,
    Prothrombin_time text,
    Vitamin_B12 text,
    Creatine_phosphokinase text,
    Ferritin text,
    Arterial_Lactic_Acid text,
    Lipase_dosage text,
    D_Dimer text,
    Albumin text,
    Hb_saturation text,
    pCO2 text,
    Base_excess text,
    pH_arterial_blood text,
    total_CO2 text,
    HCO3 text,
    pO2 text,
    Arteiral_Fio2 text,
    Phosphor text,
    ctO2 text
  
)''')

c.execute("CREATE INDEX pid_exam on exams (PID)")

In [ ]:
files = ["data/dataset.csv"]
for filename in files:
    with open(filename, mode='r') as csv_file:
          csv_reader = csv.DictReader(csv_file)
          for row in csv_reader:
             c.execute("SELECT PID FROM exams WHERE PID  = '" +row["Patient ID"] + "'")
             if c.fetchone() == None:
                c.execute("INSERT INTO exams(PID, SARSCov2, Hematocrit, Hemoglobin, Platelets, mean_platelet_volume, red_blood_cells, lymphocytes, mean_corpuscular_hemoglobin_concentration, leukocytes, basophils,mean_corpuscular_hemoglobi, eosinophils, mean_corpuscular_volume, monocytes, red_blood_cell_distribution, serum_glucose, respiratory_syncytial_virus, influenza_aa, influenza_bb, parainfluenza_1, coronavirusNL63, rhinovirus, mycoplasma_pneumoniae, coronavirus_HKU1, parainfluenza_3, chlamydophila_pneumoniae, adenovirus, parainfluenza_4, coronavirus229E, coronavirusOC43, inf_A_H1N1_2009, bordetella_pertussis, metapneumovirus, parainfluenza_2, neutrophils, urea, proteina_C_reativa, creatinine, potassium, sodium, influenza_B, influenza_A, alanine_transaminase, aspartate_transaminase,gamma_glutamyltransferase, total_bilirubin, direct_bilirubin, indirect_bilirubin, alkaline_phosphatase, ionized_calcium, strepto_A, magnesium, pCO2_v, Hb_saturation_v, base_excess_v, pO2_v, Fio2, total_CO2, pH, HCO3_v, Rods, Segmented, Promyelocytes, Metamyelocytes, Myelocytes, Myeloblasts, Urine_Esterase, Urine_Aspect, Urine_pH, Urine_Hemoglobin, Urine_Bile_pigments, Urine_Ketone_Bodies, Urine_Nitrite, Urine_Density, Urine_Urobilinogen, Urine_Protein, Urine_Sugar, Urine_Leukocytes, Urine_Crystals, Urine_Red_blood_cells, Urine_Hyaline_cylinders, Urine_Granular_cylinders, Urine_Yeasts, Urine_Color, Partial_thromboplastin, Relationship, International, Lactic_Dehydrogenase, Prothrombin_time, Vitamin_B12, Creatine_phosphokinase, Ferritin, Arterial_Lactic_Acid, Lipase_dosage, D_Dimer, Albumin, Hb_saturation,pCO2,Base_excess,pH_arterial_blood,total_CO2,HCO3,pO2,Arteiral_Fio2,Phosphor,ctO2) VALUES(?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?, ?, ?,?,?, ?)", 
                          (row["Patient ID"],
                           row["SARS-Cov-2 exam result"],
                            row["Hematocrit"],
                            row["Hemoglobin"],
                            row["Platelets"],
                            row["Mean platelet volume "],
                            row["Red blood Cells"],
                            row["Lymphocytes"],
                            row["Mean corpuscular hemoglobin concentration (MCHC)"],
                            row["Leukocytes"],
                            row["Basophils"],
                            row["Mean corpuscular hemoglobin (MCH)"],
                            row["Eosinophils"],
                            row["Mean corpuscular volume (MCV)"],
                            row["Monocytes"],
                            row["Red blood cell distribution width (RDW)"],
                            row["Serum Glucose"],
                            row["Respiratory Syncytial Virus"],
                            row["Influenza A"],
                            row["Influenza B"],
                            row["Parainfluenza 1"],
                            row["CoronavirusNL63"],
                            row["Rhinovirus/Enterovirus"],
                            row["Mycoplasma pneumoniae"],
                            row["Coronavirus HKU1"],
                            row["Parainfluenza 3"],
                            row["Chlamydophila pneumoniae"],
                            row["Adenovirus"],
                            row["Parainfluenza 4"],
                            row["Coronavirus229E"],
                            row["CoronavirusOC43"],
                            row["Inf A H1N1 2009"],
                            row["Bordetella pertussis"],
                            row["Metapneumovirus"],
                            row["Parainfluenza 2"],
                            row["Neutrophils"],
                            row["Urea"],
                            row["Proteina C reativa mg/dL"],
                            row["Creatinine"],
                            row["Potassium"],
                            row["Sodium"],
                            row["Influenza B, rapid test"],
                            row["Influenza A, rapid test"],
                            row["Alanine transaminase"],
                            row["Aspartate transaminase"],
                            row["Gamma-glutamyltransferase "],
                            row["Total Bilirubin"],
                            row["Direct Bilirubin"],
                            row["Indirect Bilirubin"],
                            row["Alkaline phosphatase"],
                            row["Ionized calcium "],
                            row["Strepto A"],
                            row["Magnesium"],
                            row["pCO2 (venous blood gas analysis)"],
                            row["Hb saturation (venous blood gas analysis)"],
                            row["Base excess (venous blood gas analysis)"],
                            row["pO2 (venous blood gas analysis)"],
                            row["Fio2 (venous blood gas analysis)"],
                            row["Total CO2 (venous blood gas analysis)"],
                            row["pH (venous blood gas analysis)"],
                            row["HCO3 (venous blood gas analysis)"],
                            row["Rods #"],
                            row["Segmented"],
                            row["Promyelocytes"],
                            row["Metamyelocytes"],
                            row["Myelocytes"],
                            row["Myeloblasts"],
                            row["Urine - Esterase"],
                            row["Urine - Aspect"],
                            row["Urine - pH"],
                            row["Urine - Hemoglobin"],
                            row["Urine - Bile pigments"],
                            row["Urine - Ketone Bodies"],
                            row["Urine - Nitrite"],
                            row["Urine - Density"],
                            row["Urine - Urobilinogen"],
                            row["Urine - Protein"],
                            row["Urine - Sugar"],
                            row["Urine - Leukocytes"],
                            row["Urine - Crystals"],
                            row["Urine - Red blood cells"],
                            row["Urine - Hyaline cylinders"],
                            row["Urine - Granular cylinders"],
                            row["Urine - Yeasts"],
                            row["Urine - Color"],
                            row["Partial thromboplastin time (PTT) "],
                            row["Relationship (Patient/Normal)"],
                            row["International normalized ratio (INR)"],
                            row["Lactic Dehydrogenase"],
                            row["Prothrombin time (PT), Activity"],
                            row["Vitamin B12"],
                            row["Creatine phosphokinase (CPK) "],
                            row["Ferritin"],
                            row["Arterial Lactic Acid"],
                            row["Lipase dosage"],
                            row["D-Dimer"],
                            row["Albumin"],
                            row["Hb saturation (arterial blood gases)"],
                            row["pCO2 (arterial blood gas analysis)"],
                            row["Base excess (arterial blood gas analysis)"],
                            row["pH (arterial blood gas analysis)"],
                            row["Total CO2 (arterial blood gas analysis)"],
                            row["HCO3 (arterial blood gas analysis)"],
                            row["pO2 (arterial blood gas analysis)"],
                            row["Arteiral Fio2"],
                            row["Phosphor"],
                            row["ctO2 (arterial blood gas analysis)"]
                          ))
conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM exams WHERE SARSCov2='positive'", conn)

In [ ]:
c.execute("DROP VIEW IF EXISTS positive_cases")
conn.commit()
c.execute('''CREATE VIEW positive_cases AS SELECT * FROM exams WHERE SARSCov2='positive' ''')

In [ ]:
c.execute("DROP TABLE IF EXISTS treatments")
conn.commit()
c.execute('''CREATE TABLE treatments(
    PID int, 
    regular text,
    semi_intensive text,
    intensive text
  
)''')

c.execute("CREATE INDEX pid_treatment on treatments (PID)")

In [ ]:
files = ["data/dataset.csv"]
for filename in files:
    with open(filename, mode='r') as csv_file:
          csv_reader = csv.DictReader(csv_file)
          for row in csv_reader:
             c.execute("SELECT PID FROM treatments WHERE PID  = '" +row["Patient ID"] + "'")
             if c.fetchone() == None:
                c.execute("INSERT INTO treatments(PID,regular,semi_intensive,intensive) VALUES(?, ?, ?, ?)", 
                          (row["Patient ID"], 
                           row["Patient addmited to regular ward (1=yes, 0=no)"], 
                           row["Patient addmited to semi-intensive unit (1=yes, 0=no)"],
                           row["Patient addmited to intensive care unit (1=yes, 0=no)"]
                          ))
conn.commit()

In [ ]:
pd.read_sql_query("SELECT * FROM treatments NATURAL JOIN positive_cases AS e WHERE e.SARSCov2='positive'", conn)

In [ ]:
pd.read_sql_query("SELECT * FROM treatments t NATURAL JOIN positive_cases AS e WHERE t.intensive='1'", conn)

In [ ]:
pd.read_sql_query("SELECT * FROM treatments t NATURAL JOIN positive_cases AS e WHERE t.regular='1'", conn)

# Data processing

In [ ]:
database = pd.read_csv('data/dataset.csv')

database = database.replace('not_detected', 0)
database = database.replace('detected', 1)
database = database.replace('negative', 0)
database = database.replace('positive', 1)
database = database.replace('not_done', 0)
database = database.replace('done', 1)
database = database.replace('absent',0)
database = database.replace('clear', 0)
database = database.replace('altered_coloring', 1)
database = database.replace('normal', 0)
database = database.replace('cloudy', 1)
database = database.replace('lightly_cloudy', 0.5)
database = database.replace('Não Realizado', 0)
database = database.replace('present', 1)
database = database.replace('<1000', 0)
database = database.replace('Ausentes', 0)
database = database.replace('Urato Amorfo --+', 0.5)
database = database.replace('Urato Amorfo +++', 1)
database = database.replace('Oxalato de Cálcio +++', 1)
database = database.replace('Oxalato de Cálcio -++', 0.5)
database = database.replace('light_yellow', 0.5)
database = database.replace('yellow', 0.7)
database = database.replace('orange', 1)
database = database.replace('citrus_yellow', 0.8)
database = database.replace([np.nan, 'NaN', 'nan'], 0)

In [ ]:
# print(database.groupby('SARS-Cov-2 exam result').size())
training_database = pd.concat([database[database["SARS-Cov-2 exam result"] == 1].head(500), database[database["SARS-Cov-2 exam result"] == 0].head(500)])
positive_count = training_database.shape[0]
testing_size = positive_count - 500

testing_database = pd.concat([database[database["SARS-Cov-2 exam result"] == 1].tail(testing_size), database[database["SARS-Cov-2 exam result"] == 0].tail(testing_size)])

# print(testing_database)

# PCA

In [ ]:
from sklearn.impute import SimpleImputer

np.random.seed(5)

centers = [[1, 1], [-1, -1], [1, -1]]
db = training_database
df = db


# print(db.where(db=='not_detected'))

feature_names = list(db.iloc[:,3:111])
# print(feature_names)

y = db['SARS-Cov-2 exam result']
# print(y)
samples = db.drop(columns=['SARS-Cov-2 exam result', 'Patient ID', 'Patient age quantile'])

imp = SimpleImputer(strategy='mean')
X = imp.fit_transform(samples.values)

# print(X)

# print(y)

print(db.groupby('SARS-Cov-2 exam result').size())

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# X = StandardScaler().fit_transform(X)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, df[['SARS-Cov-2 exam result']]], axis = 1)

# print(finalDf)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0, 1]
colors = ['r', 'g']

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['SARS-Cov-2 exam result'] == target
#     print(finalDf.loc[indicesToKeep, 'principal component 1'])
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 40)
ax.legend(targets)
ax.grid()

In [ ]:
# db_blood_exams = training_database[main_features['blood_exams'][6:]]

db = database
y = db['SARS-Cov-2 exam result']
db = db.drop(columns=['SARS-Cov-2 exam result', 'Patient ID', 'Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 
                                   'SARS-Cov-2 exam result'])

db.replace(np.nan, 0, inplace=True)

pca = decomposition.PCA(n_components=2)
pca.fit(db.values)
x_pca = pca.transform(db.values)

# fig = plt.figure(figsize = (20,20))

plt.plot(x_pca[y == 0, 0], x_pca[y == 0, 1], 'bo', label='Negativo')
plt.plot(x_pca[y == 1, 0], x_pca[y == 1, 1], 'go', label='Positivo')
plt.legend(loc=0)

# print(pca.explained_variance_ratio_)
# print(pca.singular_values_)
print(pd.DataFrame(pca.components_,columns=db.columns,index = ['PC-1','PC-2']))

In [ ]:
# from sklearn.manifold import TSNE

# tsne = TSNE(random_state=17)

# x_tsne = tsne.fit_transform(db.values)


# plt.figure(figsize=(12,10))
# plt.scatter(x_tsne[:, 0], x_tsne[:, 1], c=y, edgecolor='none',alpha=0.7, s=40)
# plt.colorbar()

In [ ]:
categories = {
  "infections": [
    'Patient age quantile',
    'SARS-Cov-2 exam result',
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'Respiratory Syncytial Virus',
    'Influenza A',
    'Influenza B',
    'Parainfluenza 1',
    'CoronavirusNL63',
    'Rhinovirus/Enterovirus',
    'Coronavirus HKU1',
    'Parainfluenza 3',
    'Chlamydophila pneumoniae',
    'Adenovirus',
    'Parainfluenza 4',
    'Coronavirus229E',
    'CoronavirusOC43',
    'Inf A H1N1 2009',
    'Bordetella pertussis',
    'Metapneumovirus',
    'Parainfluenza 2',
    'Influenza B, rapid test',
    'Influenza A, rapid test',
    'Strepto A'
  ],
  "blood": [
    'Patient age quantile',
    'SARS-Cov-2 exam result',
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'Hematocrit',
    'Hemoglobin',
    'Platelets',
    'Mean platelet volume ',
    'Red blood Cells',
    'Lymphocytes',
    'Mean corpuscular hemoglobin concentration\xa0(MCHC)',
    'Leukocytes',
    'Basophils',
    'Mean corpuscular hemoglobin (MCH)',
    'Eosinophils',
    'Mean corpuscular volume (MCV)',
    'Monocytes',
    'Red blood cell distribution width (RDW)',
    'Serum Glucose',
    'Neutrophils',
    'Urea',
    'Proteina C reativa mg/dL',
    'Creatinine',
    'Alanine transaminase',
    'Aspartate transaminase',
    'Gamma-glutamyltransferase\xa0',
    'Total Bilirubin',
    'Direct Bilirubin',
    'Indirect Bilirubin',
    'Lactic Dehydrogenase',
    'Creatine phosphokinase\xa0(CPK)\xa0',
    'Arterial Lactic Acid',
    'Lipase dosage',
    'Albumin',
    'Arteiral Fio2'
  ],
  "mineravit": [
    'Patient age quantile',
    'SARS-Cov-2 exam result',
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'Potassium',
    'Sodium',
    'Alkaline phosphatase',
    'Ionized calcium\xa0',
    'Magnesium',
    'Vitamin B12',
    'Ferritin',
    'Phosphor'
  ],
  "urine": [
    'Patient age quantile',
    'SARS-Cov-2 exam result',
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'Urine - Esterase',
    'Urine - Aspect',
    'Urine - pH',
    'Urine - Hemoglobin',
    'Urine - Bile pigments',
    'Urine - Ketone Bodies',
    'Urine - Nitrite',
    'Urine - Density',
    'Urine - Urobilinogen',
    'Urine - Protein',
    'Urine - Leukocytes',
    'Urine - Crystals',
    'Urine - Red blood cells',
    'Urine - Hyaline cylinders',
    'Urine - Granular cylinders',
    'Urine - Yeasts',
    'Urine - Color'
  ],
  "gas_analysis": [
    'Patient age quantile',
    'SARS-Cov-2 exam result',
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'pCO2 (venous blood gas analysis)',
    'Hb saturation (venous blood gas analysis)',
    'Base excess (venous blood gas analysis)',
    'pO2 (venous blood gas analysis)',
    'Fio2 (venous blood gas analysis)',
    'Total CO2 (venous blood gas analysis)',
    'pH (venous blood gas analysis)',
    'HCO3 (venous blood gas analysis)',
    'Hb saturation (arterial blood gases)',
    'pCO2 (arterial blood gas analysis)',
    'Base excess (arterial blood gas analysis)',
    'pH (arterial blood gas analysis)',
    'Total CO2 (arterial blood gas analysis)',
    'HCO3 (arterial blood gas analysis)',
    'pO2 (arterial blood gas analysis)',
    'ctO2 (arterial blood gas analysis)'
  ],
  "cancer": [
    'Patient age quantile',
    'SARS-Cov-2 exam result',
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'Promyelocytes',
    'Metamyelocytes',
    'Myelocytes',
    'Myeloblasts'
  ]
}

In [ ]:
dataset = training_database
db_infections = dataset[categories["infections"]]
db_infections.replace([np.nan, 'NaN', "negative", "not_detected"], 0, inplace=True)
db_infections.replace(["positive", "detected"], 1, inplace=True)

print(db_infections)

In [ ]:
positive_infections = db_infections[db_infections['SARS-Cov-2 exam result'] == 1]

In [ ]:
columns = positive_infections.columns
to_remove = ['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 'SARS-Cov-2 exam result']
for col in to_remove:
    columns = np.delete(columns, np.where(columns == col))

x = []
y = []

for col in columns:
    l = positive_infections[positive_infections[col] == 1]
    x.append(col)
    y.append(l.size)

y_pos = np.arange(len(y))
 
plt.bar(y_pos, y, color = '#2977b4')
plt.xlabel('Co-infections with Sars-Cov-2')
plt.ylabel('Number of people')

plt.ylim(0,max(y)+100)
plt.xticks(y_pos, x, rotation=90)
 
plt.show()

In [ ]:
db_blood = dataset[categories["blood"]].replace([np.nan, 'NaN'], 0)

In [ ]:
values_db = db_blood.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 'SARS-Cov-2 exam result'])

blood_x = StandardScaler().fit_transform(values_db.values)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(blood_x)

print(pd.DataFrame(pca.components_,columns=values_db.columns,index = ['PC-1','PC-2']))

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, db_blood[['SARS-Cov-2 exam result']]], axis = 1)

# print(finalDf)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0, 1]
colors = ['r', 'g']

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['SARS-Cov-2 exam result'] == target
#     print(finalDf.loc[indicesToKeep, 'principal component 1'])
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 20)
ax.legend(['Negative', 'Positive'])
ax.grid()

In [ ]:
db_urine = dataset[categories["urine"]].replace([np.nan, 'NaN', 'absent', 'not_done', 'Ausentes', 'clear', '<1000', 'Não Realizado', 'normal'], 0)
db_urine = db_urine.replace(['cloudy', 'altered_coloring', 'present', 'Oxalato de Cálcio +++', 'orange', 'Urato Amorfo +++'], 1)
db_urine = db_urine.replace(['lightly_cloudy', 'Urato Amorfo --+', 'light_yellow'], 0.5)
db_urine = db_urine.replace(['Oxalato de Cálcio -++', 'yellow', 'citrus_yellow'], 0.7)

values_db = db_urine.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 'SARS-Cov-2 exam result'])

urine_x = StandardScaler().fit_transform(values_db.values)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(urine_x)

print(pd.DataFrame(pca.components_,columns=values_db.columns,index = ['PC-1','PC-2']))

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, db_urine[['SARS-Cov-2 exam result']]], axis = 1)

# print(finalDf)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0, 1]
colors = ['r', 'g']

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['SARS-Cov-2 exam result'] == target
#     print(finalDf.loc[indicesToKeep, 'principal component 1'])
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(['Negative', 'Positive'])
ax.grid()

In [ ]:
db_mineravit = dataset[categories["mineravit"]].replace([np.nan, 'NaN', 'absent', 'not_done', 'Ausentes', 'clear', '<1000', 'Não Realizado', 'normal'], 0)
db_mineravit = db_mineravit.replace(['cloudy', 'altered_coloring', 'present', 'Oxalato de Cálcio +++', 'orange', 'Urato Amorfo +++'], 1)
db_mineravit = db_mineravit.replace(['lightly_cloudy', 'Urato Amorfo --+', 'light_yellow'], 0.5)
db_mineravit = db_mineravit.replace(['Oxalato de Cálcio -++', 'yellow', 'citrus_yellow'], 0.7)

values_db = db_mineravit.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 'SARS-Cov-2 exam result'])

mineravit_x = StandardScaler().fit_transform(values_db.values)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(mineravit_x)

print(pd.DataFrame(pca.components_,columns=values_db.columns,index = ['PC-1','PC-2']))

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, db_mineravit[['SARS-Cov-2 exam result']]], axis = 1)

# print(finalDf)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0,1]
colors = ['r', 'g']

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['SARS-Cov-2 exam result'] == target
#     print(finalDf.loc[indicesToKeep, 'principal component 1'])
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(['Negative', 'Positive'])
ax.grid()

In [ ]:
db_gas_analysis = dataset[categories["gas_analysis"]].replace([np.nan, 'NaN', 'absent', 'not_done', 'Ausentes', 'clear', '<1000', 'Não Realizado', 'normal'], 0)
db_gas_analysis = db_gas_analysis.replace(['cloudy', 'altered_coloring', 'present', 'Oxalato de Cálcio +++', 'orange', 'Urato Amorfo +++'], 1)
db_gas_analysis = db_gas_analysis.replace(['lightly_cloudy', 'Urato Amorfo --+', 'light_yellow'], 0.5)
db_gas_analysis = db_gas_analysis.replace(['Oxalato de Cálcio -++', 'yellow', 'citrus_yellow'], 0.7)

values_db = db_gas_analysis.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 'SARS-Cov-2 exam result'])

gas_analysis_x = StandardScaler().fit_transform(values_db.values)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(gas_analysis_x)

print(pd.DataFrame(pca.components_,columns=values_db.columns,index = ['PC-1','PC-2']))

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, db_gas_analysis[['SARS-Cov-2 exam result']]], axis = 1)

# print(finalDf)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0,1]
colors = ['r', 'g']

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['SARS-Cov-2 exam result'] == target
#     print(finalDf.loc[indicesToKeep, 'principal component 1'])
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(['Negative', 'Positive'])
ax.grid()

In [ ]:
db_cancer = dataset[categories["cancer"]].replace([np.nan, 'NaN', 'absent', 'not_done', 'Ausentes', 'clear', '<1000', 'Não Realizado', 'normal'], 0)
db_cancer = db_cancer.replace(['cloudy', 'altered_coloring', 'present', 'Oxalato de Cálcio +++', 'orange', 'Urato Amorfo +++'], 1)
db_cancer = db_cancer.replace(['lightly_cloudy', 'Urato Amorfo --+', 'light_yellow'], 0.5)
db_cancer = db_cancer.replace(['Oxalato de Cálcio -++', 'yellow', 'citrus_yellow'], 0.7)

values_db = db_cancer.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 'SARS-Cov-2 exam result'])

cancer_x = StandardScaler().fit_transform(values_db.values)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(cancer_x)

print(pd.DataFrame(pca.components_,columns=values_db.columns,index = ['PC-1','PC-2']))

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, db_cancer[['SARS-Cov-2 exam result']]], axis = 1)

# print(finalDf)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0,1]
colors = ['r', 'g']

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['SARS-Cov-2 exam result'] == target
#     print(finalDf.loc[indicesToKeep, 'principal component 1'])
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(['Negative', 'Positive'])
ax.grid()

# HEATMAPS

In [ ]:
values_db = db_blood
values_db = values_db.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 'SARS-Cov-2 exam result'])
plt.subplots(figsize=(20,15))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)
plt.show()

In [ ]:
values_db = db_blood
values_db = values_db.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)'])
values_db = values_db[values_db['SARS-Cov-2 exam result'] == 1]
plt.subplots(figsize=(8,8))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)
plt.show()

In [ ]:
values_db = db_blood
values_db = values_db.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)'])
values_db = values_db[values_db['SARS-Cov-2 exam result'] == 0]
# print(values_db.corr())
plt.subplots(figsize=(20,15))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)
plt.show()

In [ ]:
plt.figure(figsize=[20,10])

values_db = db_blood
values_db = values_db.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)'])
values_db = values_db[values_db['SARS-Cov-2 exam result'] == 1]
plt.subplot(1,2,1)
# plt.subplots(figsize=(8,8))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)


plt.subplot(1,2,2)
values_db = db_blood.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)'])
values_db = values_db[values_db['SARS-Cov-2 exam result'] == 0]
# print(values_db.corr())
# plt.subplots(figsize=(8,8))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels((),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)

# plt.tight_layout()

plt.show()

# Blood test inflammation

In [ ]:
main_features = {
    "blood_exams" : [
    'Patient ID',
    'Patient age quantile',
    'SARS-Cov-2 exam result',
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'Proteina C reativa mg/dL', #inflammation
    'Ferritin', # inflammation
    'Hemoglobin', #inflammation
#     'Red blood cell distribution width (RDW)', #inflammation
    'Neutrophils', #inflammation
    'Platelets',
    'Serum Glucose', #diabetes
    'Aspartate transaminase', #kidney or heart demage
#     'Vitamin B12', #anmeia
    'Lymphocytes', #leukemia indicator
    'Potassium', #kidney disease, high blood pressure, and heart disease
    'Urea', #how well the kidneys are working
    'Albumin', #kidney and liver problems
    'Arteiral Fio2', #difficulty to breath
    'Arterial Lactic Acid', #severe anemia or leukemia. Liver disease or liver damage that prevents the liver from breaking down lactic acid in the blood
]}

In [ ]:
db_blood_exams = training_database[main_features['blood_exams']]

values_db = db_blood
values_db = values_db.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)'])

values_db = values_db[values_db['SARS-Cov-2 exam result'] == 0]
# print(values_db.corr())
plt.subplots(figsize=(20,15))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)
plt.show()

In [ ]:
db_blood_exams = training_database[main_features['blood_exams']]

values_db = db_blood
values_db = values_db.drop(columns=['Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)'])

values_db = values_db[values_db['SARS-Cov-2 exam result'] == 1]
values_db = values_db[values_db['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1]

# print(values_db.corr())
plt.subplots(figsize=(20,15))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)
plt.show()

In [ ]:
db_blood_exams = training_database[main_features['blood_exams']]

values_db = db_blood
values_db = values_db.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)'])

values_db = values_db[values_db['SARS-Cov-2 exam result'] == 1]
values_db = values_db[values_db['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 1]

# print(values_db.corr())
plt.subplots(figsize=(20,15))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)
plt.show()

In [ ]:
db_blood_exams = training_database[main_features['blood_exams']]

values_db = db_blood
values_db = values_db.drop(columns=['Patient addmited to semi-intensive unit (1=yes, 0=no)',
        'Patient addmited to intensive care unit (1=yes, 0=no)'])

values_db = values_db[values_db['SARS-Cov-2 exam result'] == 1]
values_db = values_db[values_db['Patient addmited to regular ward (1=yes, 0=no)'] == 1]

# print(values_db.corr())
plt.subplots(figsize=(20,15))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)
plt.show()

In [ ]:
db_blood_exams = training_database[main_features['blood_exams']]

values_db = db_blood
values_db = values_db.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)'])

values_db = values_db[values_db['SARS-Cov-2 exam result'] == 1]
# print(values_db.corr())
plt.subplots(figsize=(20,15))
s=sns.heatmap(values_db.corr(),cmap='coolwarm') 
s.set_yticklabels(s.get_yticklabels(),rotation=0,fontsize=11)
s.set_xticklabels(s.get_xticklabels(),rotation=90,fontsize=11)
plt.show()

In [ ]:
db_blood_exams = training_database[main_features['blood_exams']]
values_db = db_blood
values_db = values_db.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
       'Patient addmited to semi-intensive unit (1=yes, 0=no)',
       'Patient addmited to intensive care unit (1=yes, 0=no)', 
                                   'SARS-Cov-2 exam result'])

# x = StandardScaler().fit_transform(values_db.values)

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(values_db.values)

# print(pd.DataFrame(pca.components_,columns=values_db.columns,index = ['PC-1','PC-2']))

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, db_gas_analysis[['SARS-Cov-2 exam result']]], axis = 1)

# print(finalDf)
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

targets = [0,1]
colors = ['grey', 'g']

for target, color in zip(targets,colors):
    indicesToKeep = finalDf['SARS-Cov-2 exam result'] == target
#     print(finalDf.loc[indicesToKeep, 'principal component 1'])
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(['Negative', 'Positive'])
ax.grid()

In [ ]:
db_blood_exams = training_database[main_features['blood_exams']]

# print(db_blood_exams)
 
#normal = 0, abnormal = 1, ausent = 2

# db_blood_exams['Proteina C reativa mg/dL'] = db_blood_exams['Proteina C reativa mg/dL'].mask(db_blood_exams['Proteina C reativa mg/dL'] >= 2, 1)
# db_blood_exams['Proteina C reativa mg/dL'] = db_blood_exams['Proteina C reativa mg/dL'].mask(db_blood_exams['Proteina C reativa mg/dL'] < 2, 0)
# db_blood_exams['Proteina C reativa mg/dL'] = db_blood_exams['Proteina C reativa mg/dL'].replace(np.nan, 2)

# db_blood_exams['Ferritin'] = db_blood_exams['Ferritin'].mask(db_blood_exams['Ferritin'] >= 2, 1)
# db_blood_exams['Ferritin'] = db_blood_exams['Ferritin'].mask(db_blood_exams['Ferritin'] < 2, 0)
# db_blood_exams['Ferritin'] = db_blood_exams['Ferritin'].replace(np.nan, 2)

# db_blood_exams['Hemoglobin'] = db_blood_exams['Hemoglobin'].mask(db_blood_exams['Hemoglobin'] >= 2, 1)
# db_blood_exams['Hemoglobin'] = db_blood_exams['Hemoglobin'].mask(db_blood_exams['Hemoglobin'] < 2, 0)
# db_blood_exams['Hemoglobin'] = db_blood_exams['Hemoglobin'].replace(np.nan, 2)

# db_blood_exams['Red blood cell distribution width (RDW)'] = db_blood_exams['Red blood cell distribution width (RDW)'].mask(db_blood_exams['Red blood cell distribution width (RDW)'] >= 2, 1)
# db_blood_exams['Red blood cell distribution width (RDW)'] = db_blood_exams['Red blood cell distribution width (RDW)'].mask(db_blood_exams['Red blood cell distribution width (RDW)'] < 2, 0)
# db_blood_exams['Red blood cell distribution width (RDW)'] = db_blood_exams['Red blood cell distribution width (RDW)'].replace(np.nan, 2)

# db_blood_exams['Neutrophils'] = db_blood_exams['Neutrophils'].mask(db_blood_exams['Neutrophils'] >= 2, 1)
# db_blood_exams['Neutrophils'] = db_blood_exams['Neutrophils'].mask(db_blood_exams['Neutrophils'] < 2, 0)
# db_blood_exams['Neutrophils'] = db_blood_exams['Neutrophils'].replace(np.nan, 2)

# db_blood_exams['Platelets'] = db_blood_exams['Platelets'].mask(db_blood_exams['Platelets'] >= 2, 1)
# db_blood_exams['Platelets'] = db_blood_exams['Platelets'].mask(db_blood_exams['Platelets'] < 2, 0)
# db_blood_exams['Platelets'] = db_blood_exams['Platelets'].replace(np.nan, 2)

# db_blood_exams['Serum Glucose'] = db_blood_exams['Serum Glucose'].mask(db_blood_exams['Serum Glucose'] >= 2, 1)
# db_blood_exams['Serum Glucose'] = db_blood_exams['Serum Glucose'].mask(db_blood_exams['Serum Glucose'] < 2, 0)
# db_blood_exams['Serum Glucose'] = db_blood_exams['Serum Glucose'].replace(np.nan, 2)

# db_blood_exams['Aspartate transaminase'] = db_blood_exams['Aspartate transaminase'].mask(db_blood_exams['Aspartate transaminase'] >= 2, 1)
# db_blood_exams['Aspartate transaminase'] = db_blood_exams['Aspartate transaminase'].mask(db_blood_exams['Aspartate transaminase'] < 2, 0)
# db_blood_exams['Aspartate transaminase'] = db_blood_exams['Aspartate transaminase'].replace(np.nan, 2)

# db_blood_exams['Vitamin B12'] = db_blood_exams['Vitamin B12'].mask(db_blood_exams['Vitamin B12'] >= 2, 1)
# db_blood_exams['Vitamin B12'] = db_blood_exams['Vitamin B12'].mask(db_blood_exams['Vitamin B12'] < 2, 0)
# db_blood_exams['Vitamin B12'] = db_blood_exams['Vitamin B12'].replace(np.nan, 2)

# db_blood_exams['Lymphocytes'] = db_blood_exams['Lymphocytes'].mask(db_blood_exams['Lymphocytes'] >= 2, 1)
# db_blood_exams['Lymphocytes'] = db_blood_exams['Lymphocytes'].mask(db_blood_exams['Lymphocytes'] < 2, 0)
# db_blood_exams['Lymphocytes'] = db_blood_exams['Lymphocytes'].replace(np.nan, 2)

# db_blood_exams['Potassium'] = db_blood_exams['Potassium'].mask(db_blood_exams['Potassium'] >= 2, 1)
# db_blood_exams['Potassium'] = db_blood_exams['Potassium'].mask(db_blood_exams['Potassium'] < 2, 0)
# db_blood_exams['Potassium'] = db_blood_exams['Potassium'].replace(np.nan, 2)

# db_blood_exams['Urea'] = db_blood_exams['Urea'].mask(db_blood_exams['Urea'] >= 2, 1)
# db_blood_exams['Urea'] = db_blood_exams['Urea'].mask(db_blood_exams['Urea'] < 2, 0)
# db_blood_exams['Urea'] = db_blood_exams['Urea'].replace(np.nan, 2)

# db_blood_exams['Albumin'] = db_blood_exams['Albumin'].mask(db_blood_exams['Albumin'] >= 2, 1)
# db_blood_exams['Albumin'] = db_blood_exams['Albumin'].mask(db_blood_exams['Albumin'] < 2, 0)
# db_blood_exams['Albumin'] = db_blood_exams['Albumin'].replace(np.nan, 2)

# db_blood_exams['Arteiral Fio2'] = db_blood_exams['Arteiral Fio2'].mask(db_blood_exams['Arteiral Fio2'] >= 2, 1)
# db_blood_exams['Arteiral Fio2'] = db_blood_exams['Arteiral Fio2'].mask(db_blood_exams['Arteiral Fio2'] < 2, 0)
# db_blood_exams['Arteiral Fio2'] = db_blood_exams['Arteiral Fio2'].replace(np.nan, 2)

# db_blood_exams['Arterial Lactic Acid'] = db_blood_exams['Arterial Lactic Acid'].mask(db_blood_exams['Arterial Lactic Acid'] >= 2, 1)
# db_blood_exams['Arterial Lactic Acid'] = db_blood_exams['Arterial Lactic Acid'].mask(db_blood_exams['Arterial Lactic Acid'] < 2, 0)
# db_blood_exams['Arterial Lactic Acid'] = db_blood_exams['Arterial Lactic Acid'].replace(np.nan, 2)

# print(db_blood_exams)

In [ ]:
from sklearn.decomposition import PCA, KernelPCA

y = training_database['SARS-Cov-2 exam result']
db_blood_exams = training_database[main_features['blood_exams'][6:]]

# print(db_blood_exams.columns)

db_blood_exams.replace(np.nan, 0, inplace=True)

pca = decomposition.PCA(n_components=2)
pca.fit(db_blood_exams.values)
x_pca = pca.transform(db_blood_exams.values)


# kpca = KernelPCA(kernel="rbf", fit_inverse_transform=True, gamma=10)
# X_kpca = kpca.fit_transform(db_blood_exams.values)
# X_back = kpca.inverse_transform(X_kpca)
# pca = PCA()
# x_pca = pca.fit_transform(X)

plt.plot(x_pca[y == 0, 0], x_pca[y == 0, 1], 'bo', label='Negativo')
plt.plot(x_pca[y == 1, 0], x_pca[y == 1, 1], 'go', label='Positivo')
plt.legend(loc=0)

print(pca.explained_variance_ratio_)
print(pca.singular_values_)
print(pd.DataFrame(pca.components_,columns=db_blood_exams.columns,index = ['PC-1','PC-2']))

In [ ]:
import time

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin
from sklearn.datasets import make_blobs

k_means = KMeans(init='k-means++', n_clusters=2, n_init=10)
t0 = time.time()
k_means.fit(db_blood_exams.values)
t_batch = time.time() - t0

n_clusters = 2

k_means_cluster_centers = k_means.cluster_centers_

# k_means_labels = pairwise_distances_argmin(X, k_means_cluster_centers)

# ax = fig.add_subplot(1, 3, 1)
# for k, col in zip(range(n_clusters), colors):
#     my_members = k_means_labels == k
#     cluster_center = k_means_cluster_centers[k]
#     ax.plot(X[my_members, 0], X[my_members, 1], 'w',
#             markerfacecolor=col, marker='.')
#     ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
#             markeredgecolor='k', markersize=6)
# ax.set_title('KMeans')
# ax.set_xticks(())
# ax.set_yticks(())
# plt.text(-3.5, 1.8,  'train time: %.2fs\ninertia: %f' % (
#     t_batch, k_means.inertia_))

In [ ]:
# from sklearn.manifold import TSNE

# y = training_database['SARS-Cov-2 exam result']
# db_blood_exams = training_database[main_features['blood_exams'][6:]]

# tsne = TSNE(random_state=17)

# # print(db_blood_exams.values)
# x_tsne = tsne.fit_transform(db_blood_exams.values)

# plt.figure(figsize=(12,10))
# plt.scatter(x_tsne[:, 0], x_tsne[:, 1], c=y, edgecolor='none',alpha=0.7, s=40)
# plt.colorbar()

# print(tsne.get_params())

In [ ]:
# from sklearn.manifold import TSNE


# values_db = training_database[main_features['blood_exams']]

# values_db = values_db.drop(columns=['Patient addmited to semi-intensive unit (1=yes, 0=no)', 'Patient ID',
#         'Patient age quantile', 'Patient addmited to intensive care unit (1=yes, 0=no)'])

# values_db = values_db[values_db['SARS-Cov-2 exam result'] == 1]

# y = values_db['Patient addmited to regular ward (1=yes, 0=no)']

# tsne = TSNE(random_state=17)

# x_tsne = tsne.fit_transform(values_db.values)

# plt.figure(figsize=(12,10))
# plt.scatter(x_tsne[:, 0], x_tsne[:, 1], c=y, edgecolor='none',alpha=0.7, s=40)
# plt.colorbar()

# print(tsne.get_params())

# Linear Discriminant analysis

In [ ]:
main_features = {
    "exams" : [
    'Patient ID',
#     'Patient age quantile',
    'SARS-Cov-2 exam result',
#     'Patient addmited to regular ward (1=yes, 0=no)',
#     'Patient addmited to semi-intensive unit (1=yes, 0=no)',
#     'Patient addmited to intensive care unit (1=yes, 0=no)',
#     'Hematocrit',
#     'Hemoglobin',
#     'Platelets',
#     'Mean platelet volume ',
#     'Red blood Cells',
#     'Lymphocytes',
#     'Mean corpuscular hemoglobin concentration\xa0(MCHC)',
#     'Leukocytes',
#     'Basophils',
#     'Mean corpuscular hemoglobin (MCH)',
#     'Eosinophils',
#     'Mean corpuscular volume (MCV)',
#     'Monocytes',
#     'Red blood cell distribution width (RDW)',
#     'Serum Glucose',
#     'Neutrophils',
#     'Urea',
#     'Proteina C reativa mg/dL',
#     'Creatinine',
#     'Alanine transaminase',
#     'Aspartate transaminase',
#     'Gamma-glutamyltransferase\xa0',
#     'Total Bilirubin',
#     'Direct Bilirubin',
#     'Indirect Bilirubin',
#     'Lactic Dehydrogenase',
#     'Creatine phosphokinase\xa0(CPK)\xa0',
#     'Arterial Lactic Acid',
#     'Lipase dosage',
#     'Albumin',
#     'Arteiral Fio2',   
        
#     'Urine - Esterase',
#     'Urine - Aspect',
#     'Urine - pH',
#     'Urine - Hemoglobin',
#     'Urine - Bile pigments',
#     'Urine - Ketone Bodies',
#     'Urine - Nitrite',
#     'Urine - Density',
#     'Urine - Urobilinogen',
#     'Urine - Protein',
#     'Urine - Leukocytes',
#     'Urine - Crystals',
#     'Urine - Red blood cells',
#     'Urine - Hyaline cylinders',
#     'Urine - Granular cylinders',
#     'Urine - Yeasts',
#     'Urine - Color',
        
    'pCO2 (venous blood gas analysis)',
    'Hb saturation (venous blood gas analysis)',
    'Base excess (venous blood gas analysis)',
    'pO2 (venous blood gas analysis)',
    'Fio2 (venous blood gas analysis)',
    'Total CO2 (venous blood gas analysis)',
    'pH (venous blood gas analysis)',
    'HCO3 (venous blood gas analysis)',
    'Hb saturation (arterial blood gases)',
    'pCO2 (arterial blood gas analysis)',
    'Base excess (arterial blood gas analysis)',
    'pH (arterial blood gas analysis)',
    'Total CO2 (arterial blood gas analysis)',
    'HCO3 (arterial blood gas analysis)',
    'pO2 (arterial blood gas analysis)',
    'ctO2 (arterial blood gas analysis)'
    ]}

In [ ]:
from sklearn.preprocessing import StandardScaler

values_db = training_database[main_features['exams']]
y = values_db['SARS-Cov-2 exam result']
X_train = values_db.drop(columns=['Patient ID', 'SARS-Cov-2 exam result'])

# X_train = X_train.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
#        'Patient addmited to semi-intensive unit (1=yes, 0=no)',
#        'Patient addmited to intensive care unit (1=yes, 0=no)'])

X_test = training_database[main_features['exams']]
y_test = testing_database['SARS-Cov-2 exam result']
X_test = X_test.drop(columns=['Patient ID', 'SARS-Cov-2 exam result'])

# X_test = X_test.drop(columns=['Patient age quantile', 'Patient addmited to regular ward (1=yes, 0=no)',
#        'Patient addmited to semi-intensive unit (1=yes, 0=no)',
#        'Patient addmited to intensive care unit (1=yes, 0=no)'])

sc = StandardScaler()
X_train = sc.fit_transform(X_train.values)

# print(X_train)

X_test = sc.transform(X_test)


In [ ]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
# from sklearn.feature_selection import SelectFromModel

# lda = LDA(n_components=1, solver='eigen')
# # model = SelectFromModel(lda)
# # lda.fit_transform(X_train, y)
# X_train = lda.fit_transform(X_train, y)
# X_test = lda.transform(X_test)

# # print(X_train.shape)

# y_pred = lda.predict(X_test)

# print('Accuracy:' + str(accuracy_score(X_test, y_pred)))
# # print(model.estimator_.coef_)
# # print(model.get_support(indices=True))
# # print(X_train.shape)

# # print(lda.get_params())
# # print(lda.predict(X_train))
# # print(lda.explained_variance_ratio_)
# # print(lda.coef_)
# # print(lda.covariance_)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(n_estimators=100)
clf = clf.fit(X_train, y)
clf.feature_importances_  

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

Y_pred = clf.predict(X_test)

cm = confusion_matrix(y_test, Y_pred)
print(cm)
print('Accuracy:' + str(accuracy_score(y_test, Y_pred)))

In [ ]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X_train)
X_new.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC

values_db = training_database[main_features['exams']]
y = values_db['SARS-Cov-2 exam result']
X_train = values_db.drop(columns=['Patient ID', 'SARS-Cov-2 exam result'])

clf = Pipeline([('feature_selection', SelectFromModel(LinearSVC(random_state=0, tol=1e-02), threshold=-np.inf)),
  ('classification', RandomForestClassifier(n_estimators=200))
])

clf.fit(X_train, y)

Y_pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print('Accuracy:' + str(accuracy_score(y_test, Y_pred)))

print(clf.steps[1][1].feature_importances_)
selected_feat = X_train.columns[(clf.steps[0][1].get_support())]
print(selected_feat)

pd.Series(clf.steps[1][1].feature_importances_.ravel()).hist()

In [ ]:

# values_db = training_database[main_features['exams']]
# y = values_db['SARS-Cov-2 exam result']
# X_train = values_db.drop(columns=['Patient ID', 'SARS-Cov-2 exam result'])

# X_test = training_database[main_features['exams']]
# y_test = testing_database['SARS-Cov-2 exam result']
# X_test = X_test.drop(columns=['Patient ID', 'SARS-Cov-2 exam result'])

classifier = RandomForestClassifier(max_depth=10, random_state=0)

classifier.fit(X_train, y)
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy:' + str(accuracy_score(y_test, y_pred)))

In [ ]:
clf = svm.SVC(kernel='linear')

clf.fit(X_train, y)

Y_pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, Y_pred)
print(cm)
print('Accuracy:' + str(accuracy_score(y_test, Y_pred)))

In [ ]:
main_features = {
    "exams" : [
    'Patient ID',
#     'Patient age quantile',
    'SARS-Cov-2 exam result',
    'Patient addmited to regular ward (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'Proteina C reativa mg/dL',
    'Hemoglobin',
    'Red blood cell distribution width (RDW)', 
    'Neutrophils',
    'Lymphocytes',
    'Urea',
    'Platelets',   
    ]}

In [ ]:
from sklearn.neural_network import MLPClassifier

v1 = 'Patient addmited to regular ward (1=yes, 0=no)'
v2 = 'Patient addmited to semi-intensive unit (1=yes, 0=no)'
v3 = 'Patient addmited to intensive care unit (1=yes, 0=no)'

values_db = training_database[main_features['exams']]


# values_db = values_db[(values_db[v1] == 1) | (values_db[v2] == 1) | (values_db[v3] == 1)]
print(len(values_db))
y = values_db['SARS-Cov-2 exam result']
X_train = values_db.drop(columns=['Patient ID', 'SARS-Cov-2 exam result', 
                                  'Patient addmited to regular ward (1=yes, 0=no)',
                                  'Patient addmited to semi-intensive unit (1=yes, 0=no)',
                                   'Patient addmited to intensive care unit (1=yes, 0=no)'])


X_test = training_database[main_features['exams']]
y_test = testing_database['SARS-Cov-2 exam result']
X_test = X_test.drop(columns=['Patient ID', 'SARS-Cov-2 exam result', 
                              'Patient addmited to regular ward (1=yes, 0=no)',
                              'Patient addmited to semi-intensive unit (1=yes, 0=no)',
                              'Patient addmited to intensive care unit (1=yes, 0=no)'])


classifier = MLPClassifier(solver='lbfgs', alpha=1e-8, hidden_layer_sizes=(40, 10), random_state=1, max_iter=900)
classifier.fit(X_train, y)

y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy:' + str(accuracy_score(y_test, y_pred)))